In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjugJibD5UgjChTNIWPJ6IAvFa85DgCb0k-ndj6v82dvhRH932KbC0
Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save
!ls

/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save
id26.0.txt  id32.0.txt	id6.0.txt


###Gán nhãn vượt hay không vượt

In [2]:
def linear_equation(x1,y1,x2,y2):
    a = (y1-y2)/(x1-x2)
    c = y1-a*x1
    b = -1
    return a, b, c

In [3]:
def safe(value1, value2):
    if value1*value2>=0:
        return True
    return False

In [4]:
#Hà Nam
#Đường thẳng phân biệt vượt 
x1 = 935.2941176470588
y1 = 725
x2 = 1752.941176470588
y2 = 726

# Điểm để phân loại
X = 1350
Y = 500

# Đường thẳng phân chia
a1, b1, c1 = linear_equation(x1,y1,x2,y2)

In [5]:
def line(x1, y1, x2, y2):
  a = y2 - y1
  b = x1 - x2
  c = -a*x1 - b*y1
  return a, b, c

In [ ]:
line(x1,y1,x2,y2)

(1, -817.6470588235293, 591858.8235294116)

In [ ]:
a1, b1, c1

(0.0012230215827338132, -1, 723.8561151079136)

In [6]:
#Thái Nguyên
#Đường thẳng phân biệt vượt 
x3 = 803.0
y3 = 582.0
x4 = 2541.0
y4 = 583.0

# Điểm để phân loại
X1 = 1360
Y1 = 400

# Đường thẳng phân chia
a2, b2, c2 = linear_equation(x3,y3,x4,y4)

In [7]:
#Vũng Tàu
#Đường thẳng phân biệt vượt 
x5 = 807.0
y5 = 658.0
x6 = 2361.0
y6 = 659.0

# Điểm để phân loại
X2 = 1360
Y2 = 400

# Đường thẳng phân chia
a3, b3, c3 = linear_equation(x5,y5,x6,y6)

In [ ]:
path = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'
raw_label = path.split('/')[-2]
raw_label

'hanam_14.0.avi_save'

###Data loader

In [8]:
import os
import math
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

class MyCustomDataset(Sequence):
    def __init__(self, info_files, batch_size=1, max_len=147):
        """Instaniate a custom dataset.

        Attributes
          info_files (list or str): List of info files or only one file. Each file
            should have pattern like 'DendoX/*/info.txt' or 'DenxanhX/*/info.txt'.
          batch_size (int): Batch size.
          max_len (int): Maximum of length of sequence.
        """
        self.batch_size = batch_size
        self.max_len = max_len
        if isinstance(info_files, str):
            info_files = [info_files]
        
        for path in info_files:
            assert os.path.isfile(path), f'Not found info file: {path}'
        self.data = self._load_data(info_files)
    
    def on_epoch_end(self):
        np.random.shuffle(self.data)

    def _load_data(self, file_paths):
        data = []
        for path in file_paths:
            # Load label from file path.
           
            # Store object moving states in a list. So this should be a list of
            # 4-value list.
            sequences = []
            central_list = []
            bbox_list = []
            with open(path) as f:
                for line in f:
                    line = line.strip()
                    row = line.split(',')
                    #row = list(map(float, row))  # convert to float
                    bboxes = row[-4:]
                    #store coordinate central point
                    central = []
                    central.append((float(bboxes[0])+float(bboxes[1]))/2)
                    central.append((float(bboxes[2])+float(bboxes[3]))/2)  
                    #normalize bbox
                    """
                    bboxes_normalize = []
                    bboxes_normalize.append(float(bboxes[0])/2560)
                    bboxes_normalize.append(float(bboxes[1])/1920)
                    bboxes_normalize.append(float(bboxes[2])/2560)
                    bboxes_normalize.append(float(bboxes[3])/1920)
                    """
                    sequences.append(bboxes)
                    central_list.append(central)
                    bbox_list.append(bboxes)
            #process label
            name_province = path.split('/')[-2]
            check = 0
            if 'hanam' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 797:
                    check += 1
            if 'thainguyen' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 839:
                    check += 1
            if 'vungtau' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 727:
                    check += 1
            if (check >= 1): 
              label = 1 #vuot den do
            if (check == 0):
              label = 0 #khong vuot
            #padding 
            #sequences_pad = keras.preprocessing.sequence.pad_sequences([sequences], maxlen=147, padding="pre")
            while len(sequences) < 147:
              sequences.insert(0, [0,0,0,0])

            sequences_pad = sequences[:self.max_len]
            data.append((sequences_pad, label))
        return data
    
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)
    
    def __getitem__(self, index):
        """Returns a batch of shape (N, S, 4)"""
        batch_data = self.data[index:self.batch_size:(index+1)*self.batch_size]
        X_batch = np.empty((self.batch_size, self.max_len, 4))
        y_batch = []
        for sequences, label in batch_data:
            X_batch[:len(sequences), :, :] =  np.array(sequences, dtype='float32')
            y_batch.append(label)
        X_batch = np.array(X_batch, dtype='float32')
        y_batch = np.array(y_batch, dtype='int32')
        return X_batch, y_batch

In [9]:
# sắp xếp tên thư mục giống trên window
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [10]:
path = '/content/drive/MyDrive/Project/dataset/dendo/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['hanam_14.0.avi_save',
 'hanam_20.0.avi_save',
 'hanam_23.0.avi_save',
 'hanam_26.0.avi_save',
 'hanam_46.0.avi_save',
 'hanam_52.0.avi_save',
 'hanam_55.0.avi_save',
 'hanam_72.0.avi_save',
 'hanam_75.0.avi_save',
 'hanam_81.0.avi_save',
 'hanam_101.0.avi_save',
 'thainguyen_0.0.avi_save',
 'thainguyen_3.0.avi_save',
 'thainguyen_4.0.avi_save',
 'thainguyen_7.0.avi_save',
 'thainguyen_8.0.avi_save',
 'thainguyen_11.0.avi_save',
 'thainguyen_12.0.avi_save',
 'thainguyen_15.0.avi_save',
 'thainguyen_18.0.avi_save',
 'thainguyen_19.0.avi_save',
 'thainguyen_22.0.avi_save',
 'thainguyen_23.0.avi_save',
 'thainguyen_25.0.avi_save',
 'thainguyen_26.0.avi_save',
 'thainguyen_30.0.avi_save',
 'thainguyen_33.0.avi_save',
 'thainguyen_34.0.avi_save',
 'thainguyen_36.0.avi_save',
 'thainguyen_37.0.avi_save',
 'thainguyen_40.0.avi_save',
 'thainguyen_41.0.avi_save',
 'thainguyen_44.0.avi_save',
 'thainguyen_45.0.avi_save',
 'thainguyen_51.0.avi_save',
 'thainguyen_52.0.avi_save',
 'thainguyen_55

In [11]:
dataset = []
label = []
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)
"""
info_files = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'
dataset = MyCustomDataset(info_files)
for batch in dataset:
    print(batch[0].shape, batch[1].shape)
    break

print(batch[0])
"""

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

"\ninfo_files = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'\ndataset = MyCustomDataset(info_files)\nfor batch in dataset:\n    print(batch[0].shape, batch[1].shape)\n    break\n\nprint(batch[0])\n"

In [ ]:
a = [[1,2,3,4],[1,2,3,4]]
a.append([0,0,0,0])
a = keras.preprocessing.sequence.pad_sequences([a], maxlen=147, padding="post")
a
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(a)
print(masked_output)
print(masked_output._keras_mask)

In [12]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)

In [13]:
print(data.shape)
print(label_data.shape)

(253, 147, 4)
(253,)


In [ ]:
data[0]

array([[ 681.875 , 1421.787 ],
       [ 683.281 , 1421.698 ],
       [ 686.875 , 1421.759 ],
       [ 681.875 , 1422.912 ],
       [ 683.75  , 1424.267 ],
       [ 678.281 , 1423.8676],
       [ 678.7525, 1422.3055],
       [ 677.031 , 1420.6245],
       [ 677.969 , 1420.948 ],
       [ 678.906 , 1420.0885],
       [ 678.75  , 1419.403 ],
       [ 678.281 , 1419.3375],
       [ 678.7475, 1419.08  ],
       [ 677.5   , 1418.6794],
       [ 677.5   , 1419.002 ],
       [ 677.4975, 1418.388 ],
       [ 677.4975, 1418.5615],
       [ 678.1275, 1418.486 ],
       [ 677.5   , 1418.426 ],
       [ 677.5025, 1418.436 ],
       [ 678.125 , 1418.741 ],
       [ 677.5   , 1418.3055],
       [ 677.5025, 1418.5145],
       [ 677.969 , 1418.793 ],
       [ 678.281 , 1418.6865],
       [ 677.344 , 1418.9865],
       [ 676.875 , 1418.0485],
       [ 682.656 , 1418.1045],
       [ 683.1275, 1417.883 ],
       [ 682.5   , 1417.7485],
       [ 682.656 , 1417.562 ],
       [ 683.1275, 1417.583 ],
       [

In [ ]:
label

[array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], 

In [14]:
path = '/content/drive/MyDrive/Project/dataset/denxanh/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['thainguyen_0.0.avi_save',
 'thainguyen_3.0.avi_save',
 'thainguyen_4.0.avi_save',
 'thainguyen_7.0.avi_save',
 'thainguyen_8.0.avi_save',
 'thainguyen_11.0.avi_save',
 'thainguyen_12.0.avi_save',
 'thainguyen_14.0.avi_save',
 'thainguyen_15.0.avi_save',
 'thainguyen_18.0.avi_save',
 'thainguyen_19.0.avi_save',
 'thainguyen_22.0.avi_save',
 'thainguyen_23.0.avi_save',
 'thainguyen_25.0.avi_save',
 'thainguyen_26.0.avi_save',
 'thainguyen_30.0.avi_save',
 'thainguyen_33.0.avi_save',
 'thainguyen_34.0.avi_save',
 'thainguyen_36.0.avi_save',
 'thainguyen_37.0.avi_save',
 'thainguyen_40.0.avi_save',
 'thainguyen_41.0.avi_save',
 'thainguyen_44.0.avi_save',
 'thainguyen_45.0.avi_save',
 'thainguyen_51.0.avi_save',
 'thainguyen_52.0.avi_save',
 'thainguyen_55.0.avi_save',
 'thainguyen_56.0.avi_save',
 'thainguyen_59.0.avi_save',
 'thainguyen_62.0.avi_save',
 'thainguyen_63.0.avi_save',
 'thainguyen_66.0.avi_save',
 'thainguyen_67.0.avi_save',
 'vungtau_1.0.avi_save',
 'vungtau_12.0.avi_save

In [15]:
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

In [16]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)
print(data.shape)
print(label_data.shape)


(849, 147, 4)
(849,)


In [ ]:
label

[array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], 

In [ ]:
cnt_0 = 0
cnt_1 = 0
for i in label:
  if i == 0:
    cnt_0 += 1
  else:
    cnt_1 += 1
print(cnt_0)
print(cnt_1)

212
637


In [ ]:
data_final = np.concatenate([data, label_data], axis=1) 

ValueError: ignored

####Ghi ra file csv

In [17]:
#ghi ra file data
import csv
b = open('/content/drive/MyDrive/Project/data.csv', 'w')
a = csv.writer(b)
a.writerows(data)
b.close()

In [18]:
#ghi ra file label
np.savetxt('/content/drive/MyDrive/Project/label.csv', label_data, delimiter=",", fmt='%d')